In [1]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv   # GCN 패키지

In [3]:
from torch_geometric.datasets import Planetoid

dataset =  Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

Processing...
Done!


In [4]:
print(data.x.shape)
print(data.edge_index.shape)
print(dataset.num_node_features)

torch.Size([2708, 1433])
torch.Size([2, 10556])
1433


In [5]:
# 모델 구조

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()                                      # torch.nn.Module 클래스의 속성들을 가지고옴
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):                                    # Class GCN에 해당하는 객체를 데이터와 함께 호출하면 자동으로 실행됨
        x, edge_index = data.x, data.edge_index                 # Edge_index는 2행으로 구성된 출력으로, 같은 열에 있는 값들이 연결되어 있음을 나타냄
        x = self.conv1(x, edge_index)                           # 1433개의 특성을 가진 x 데이터와 연결 정보를 나타내는 edge_index를 함께 넣어 노드의 특성을 업데이트
        x = F.relu(x)
        x = F.dropout(x, training=self.training)                # self.training이 model.train()의 경우 True, model.eval()인 경우 False로 자동으로 만들어줌
        x = self.conv2(x, edge_index)                           # 16개의 특성을 가진 x 데이터와 연결 정보를 나태내는 edge_index를 함께 넣어 노드의 특성을 업데이트

        return F.log_softmax(x, dim=1)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [7]:
model.train()                                                                   # model을 학습모드로 설정

for epoch in range(100):                                                        # 100번의 에포크
    optimizer.zero_grad()                                                       # 각 에포크마다 이전에 계산된 grad를 초기화
    out = model(data)                                                           # Class GCN에 있는 forward 자동 수행, 순전파
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])            # Loss 계산, train_mask는 학습하기 위해 사용하는 노드를 의미하며 지정되어 있음
    loss.backward()                                                             # 역전파 계산, 파라미터 별 grad가 계산됨
    optimizer.step()

In [8]:
model.eval()                                                        # model을 평가모드로 설정
pred = model(data).argmax(dim=1)                                    # argmax를 통해 Classification
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()    # test_mask는 테스트하기 위해 사용하는 노드를 의미하며 지정되어 있음
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7970


In [9]:
pip freeze

absl-py==1.4.0
aiohttp==3.9.1
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.3.1
altair==4.2.2
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.1.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.11.2
bidict==0.22.1
bigframes==0.17.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.2
bqplot==0.12.42
branca==0.7.0
build==1.0.3
CacheControl==0.13.1
cachetools==5.3.2
catalogue==2.0.10
certifi==2023.11.17
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.7
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==41.0.7
cufflinks==0.17.3
cupy-cuda12x==12.2.0
cvxopt==1.3.2
cvxpy==1.3.2
cycler==0.12.1
c